# Reproduce FID and Directional loss
Please run the notebook below to reproduce the metrics reported in the blogpost.

In [ ]:
import json
import os
import sys
from glob import glob
from pathlib import Path

import numpy as np
from PIL import Image

sys.path.append("../../src/lib/")
sys.path.append("../../src/lib/asyrp")

from lib_utils.metrics import DirectionalSimilarity, calculate_fid
from transformers import (CLIPImageProcessor, CLIPTextModelWithProjection,
                          CLIPTokenizer, CLIPVisionModelWithProjection)
from utils.text_dic import SRC_TRG_TXT_DIC

# see text in text_dic.py
GUID = "pixar"
DEVICE = "cuda:0"

(src_texts, target_texts) = SRC_TRG_TXT_DIC[GUID]

head_abl_path = Path("/home/parting/master_AI/DL2/DL2-2023-group-15/src/eval_runs/heads_ablation/")
layer_abl_path = Path("/home/parting/master_AI/DL2/DL2-2023-group-15/src/eval_runs/layertype_ablation/")
basepaths = [head_abl_path]
results = {}

clip_id = "openai/clip-vit-large-patch14"
tokenizer = CLIPTokenizer.from_pretrained(clip_id)
text_encoder = CLIPTextModelWithProjection.from_pretrained(clip_id).to(DEVICE)
image_processor = CLIPImageProcessor.from_pretrained(clip_id)
image_encoder = CLIPVisionModelWithProjection.from_pretrained(clip_id).to(DEVICE)

dir_similarity = DirectionalSimilarity(tokenizer, text_encoder, image_processor, image_encoder)

for basepath in basepaths:
    print(f'walking from: {basepath}')
    for path in os.walk(basepath, topdown=False):

        # find ablation paths
        if "40/edited" in str(path):

            path = path[0] + "/"
            # path magic
            epoch = path.split("/40")[0][-1]
            if epoch != '1':
                continue
            ablation_type = path.split("eval_runs/")[-1]
            ablation_type = ablation_type.split("/40")[0]
            ablation_type, epoch = ablation_type[:-2], ablation_type[-1:]
            
            if not "conv" in ablation_type:
                continue

            print(f"parsing: {path}")
            path_edited = path
            path_recon = path.replace("edited", "reconstructed")
            path_original = path.replace("edited", "original")

            list_original_path = glob(path_original + "/*.png")

            scores_or = []
            scores_oe = []
            scores_re = []

            for i in range(len(list_original_path)):
                img_original_path = str(Path(path_original) / f"test_{i}_19_ngen40_original.png")
                img_reconstructed_path = str(Path(path_recon) / f"test_{i}_19_ngen40_reconstructed.png")
                img_edited_path = str(Path(path_edited) / f"test_{i}_19_ngen40_edited.png")
                
                original_image = Image.open(img_original_path)
                reconstructed_image = Image.open(img_reconstructed_path)
                edited_image = Image.open(img_edited_path)
                
                similarity_score_or = dir_similarity(original_image, reconstructed_image, src_texts, src_texts)
                similarity_score_oe = dir_similarity(original_image, edited_image, src_texts, target_texts)
                similarity_score_re = dir_similarity(reconstructed_image, edited_image, src_texts, target_texts)

                scores_or.append(float(similarity_score_or.detach().cpu()))
                scores_oe.append(float(similarity_score_oe.detach().cpu()))
                scores_re.append(float(similarity_score_re.detach().cpu()))

            sdir_or = np.mean(scores_or)
            sdir_or_var = np.std(scores_or)
            sdir_oe = np.mean(scores_oe)
            sdir_oe_var = np.std(scores_oe)
            sdir_re = np.mean(scores_re)
            sdir_re_var = np.std(scores_re)
            print("=" * 50)
            print(f"Attribute {GUID} gives original-reconstructed CLIP directional similarity: {sdir_or} ({sdir_or_var})")
            print(f"Attribute {GUID} gives original-edited CLIP directional similarity: {sdir_oe}, ({sdir_oe_var})")
            print(f"Attribute {GUID} gives reconstructed-edited CLIP directional similarity: {sdir_re} ({sdir_re_var}) ")

            score_er = calculate_fid(path_edited, path_recon)
            score_ro = calculate_fid(path_original, path_recon)
            score_eo = calculate_fid(path_edited, path_original)
            print("calculating FID")

            print(f"ablation: '{ablation_type}' on epoch {epoch}, gives FID: {score_er} between edited and reconstructed")
            print(f"ablation: '{ablation_type}' on epoch {epoch}, gives FID: {score_eo} between edited and original")
            print(f"ablation: '{ablation_type}' on epoch {epoch}, gives FID: {score_ro} between reconstructed and original")
            print("-" * 50)

            run_results = {
                "sdir_or": float(sdir_or),
                "sdir_or_var": float(sdir_or_var),
                "sdir_oe": float(sdir_oe),
                "sdir_oe_var": float(sdir_oe_var),
                "sdir_re": float(sdir_re),
                "sdir_re_var": float(sdir_re_var),
                "fid_edited_reconstructed": score_er,
                "fid_edited_original": score_eo,
                "fid_reconstructed_original": score_ro,
                "epochs": epoch,
                "ablation_name": ablation_type,
                "attr": GUID
            }
            
            results[f"{ablation_type}_{epoch}_{GUID}"] = run_results

In [2]:
# write results to json file
with open("metrics.json", "w") as f:
    f.write(json.dumps(results, indent=4))
    results